In [1]:
import json
import networkx as nx
import os
import pandas as pd
import sys

sys.path.append(os.path.abspath("../"))
from utils.serialize_graph import convert_graph_to_serializable

# Load the full graph

In [2]:
with open('../graph/unweighted_graph.json', 'r') as f:
    graph_data = json.load(f)

G_original = nx.node_link_graph(graph_data)
print("Nodes:", len(G_original.nodes))
repo_urls = [x for x in G_original.nodes]
print("Edges:", len(G_original.edges))

Nodes: 4303
Edges: 9896


# Grab high-level metrics for all relevant repos

- Subscribe to the OSO Production dataset on BigQuery (see docs [here](https://docs.opensource.observer/docs/get-started/bigquery))
- Run the query below and export it to a CSV file
- Metrics and metadata:
   - Repo path
   - Language
   - Is fork?
   - Created at
   - Updated at
   - Stars
   - Forks
   - Maintainer of (list of packages)
   - Grant recipient of (list of packages)
   - Used by (list of dependents in OSO)
   - Percentile rank (for language)   

In [3]:
from google.cloud import bigquery

# replace with your path to credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../oso_gcp_credentials.json'

# replace with your project name
client = bigquery.Client(project='opensource-observer')

In [4]:
def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

query = f"""
WITH repos AS (
  SELECT *
  FROM `oso.repositories_v0`
),
package_owners AS (
  SELECT
    package_owner_artifact_id,
    package_artifact_source,
    package_artifact_name,
    CONCAT(package_artifact_source, '/', package_artifact_name) AS package_tag
  FROM `oso.package_owners_v0`
  WHERE package_owner_artifact_id IN (SELECT artifact_id FROM repos)
),
oso_dependents AS (
  SELECT
    package_owners.package_owner_artifact_id,
    COUNT(DISTINCT package_owners.package_tag) AS num_packages,
    COUNT(DISTINCT sboms.from_artifact_namespace) AS num_dependents_in_oso,
    ARRAY_AGG(DISTINCT package_owners.package_tag) AS list_of_packages,
    ARRAY_AGG(DISTINCT sboms.from_artifact_namespace) AS list_of_dependents_in_oso
  FROM `oso.sboms_v0` AS sboms
  JOIN package_owners
    ON sboms.to_package_artifact_name = package_owners.package_artifact_name
    AND sboms.to_package_artifact_source = package_owners.package_artifact_source
  GROUP BY 1
),
grants AS (
  SELECT
    funding.to_project_id AS project_id,
    ARRAY_AGG(DISTINCT projects.display_name) AS list_of_funders,
    SUM(funding.amount) AS total_funding_usd,
    SUM(CASE WHEN funding.time > '2023-01-01' THEN funding.amount ELSE 0 END) AS total_funding_usd_since_2023
  FROM `oso.oss_funding_v0` AS funding
  JOIN `oso.projects_v1` AS projects
    ON funding.from_project_id = projects.project_id
  WHERE funding.from_project_name IN ('gitcoin', 'octant-golemfoundation', 'opencollective', 'optimism')
  GROUP BY 1
),
combined AS (
  SELECT
    repos.artifact_url AS repo_url,
    repos.artifact_namespace AS maintainer,
    repos.language,
    repos.is_fork,
    DATE(repos.created_at) as created_at,
    DATE(repos.updated_at) as updated_at,    
    repos.star_count, 
    repos.fork_count,
    COALESCE(oso_dependents.num_packages, 0) AS num_packages,
    COALESCE(oso_dependents.num_dependents_in_oso, 0) AS num_dependents_in_oso,
    oso_dependents.list_of_dependents_in_oso,
    oso_dependents.list_of_packages,
    grants.list_of_funders,
    COALESCE(grants.total_funding_usd, 0) AS total_funding_usd,
    COALESCE(grants.total_funding_usd_since_2023, 0) AS total_funding_usd_since_2023
  FROM repos
  LEFT JOIN oso_dependents
    ON repos.artifact_id = oso_dependents.package_owner_artifact_id
  LEFT JOIN grants
    ON repos.project_id = grants.project_id
)
SELECT
  *,
  PERCENT_RANK() OVER (ORDER BY num_dependents_in_oso) AS oso_dependency_rank,
  COUNT(*) OVER (PARTITION BY language) AS num_repos_in_same_language,  
  PERCENT_RANK() OVER (PARTITION BY language ORDER BY num_dependents_in_oso) AS oso_dependency_rank_for_language
FROM combined
WHERE repo_url IN ({stringify_array(repo_urls)})

"""

# results = client.query(query)
# df = results.to_dataframe()
# df.to_parquet('../datasets/oso/repo_metrics_and_metadata.parquet')

df = pd.read_parquet('../datasets/oso/repo_metrics_and_metadata.parquet')
df.tail(5)

,repo_url,maintainer,language,is_fork,created_at,updated_at,star_count,fork_count,num_packages,num_dependents_in_oso,list_of_dependents_in_oso,list_of_packages,list_of_funders,total_funding_usd,total_funding_usd_since_2023,oso_dependency_rank,num_repos_in_same_language,oso_dependency_rank_for_language
1726,https://github.com/qix-/color-convert,qix-,JavaScript,False,2011-06-10,2024-12-19,748,96,1,1761,"[bcnmy, g7dao, yearn, 0xparc, rarimo, anyswap,...",[NPM/color-convert],[Open Collective],4813.94,4367.4,0.996532,913,0.995614
1727,https://github.com/colorjs/color-name,colorjs,JavaScript,False,2014-11-23,2024-11-13,119,30,1,1764,"[5afe, aragon, lexdao, zondax, api3dao, exactl...",[NPM/color-name],[Open Collective],120.00,0.0,0.997688,913,0.996711
1728,https://github.com/lodash/lodash,lodash,JavaScript,False,2012-04-07,2024-12-19,59970,7041,238,1766,"[okx, hegic, cyfrin, daobox, docnow, efforg, e...","[NPM/lodash.camelcase, NPM/lodash.times, NPM/l...",[],0.00,0.0,0.998266,913,0.997807
1729,https://github.com/debug-js/debug,debug-js,JavaScript,False,2011-11-29,2024-12-19,11177,946,1,1766,"[ipfs, ssbc, 1inch, bcnmy, aragon, lexdao, lib...",[NPM/debug],[Open Collective],9107.20,4228.2,0.998266,913,0.997807
1730,https://github.com/isaacs/inherits,isaacs,JavaScript,False,2011-04-07,2024-12-11,353,86,1,1771,"[0xfacet, binance, crustio, edendao, zama-ai, ...",[NPM/inherits],[],0.00,0.0,0.999422,913,1.000000


# Add metrics just for the pruned version of the graph

In [5]:
with open('../graph/unweighted_graph_pruned_version.json', 'r') as f:
    pruned_graph_data = json.load(f)

G_pruned = nx.node_link_graph(pruned_graph_data)
print("Nodes:", len(G_pruned.nodes))
pruned_repo_urls = [x for x in G_pruned.nodes]
print("Edges:", len(G_pruned.edges))

Nodes: 245
Edges: 434


In [6]:
def underscore_to_camelcase(string):
    words = string.split('_')
    return words[0] + ''.join(word.title() for word in words[1:])

metrics_df = (
    df
    .set_index('repo_url')
    .drop(columns=['maintainer', 'list_of_dependents_in_oso', 'list_of_packages'])
    .assign(list_of_funders=lambda x: list(x['list_of_funders']))
)

for node in G_pruned.nodes():
    if node in metrics_df.index:
        G_pruned.nodes[node]['status'] = 'indexed'
        for col in metrics_df.columns:
            G_pruned.nodes[node][underscore_to_camelcase(col)] = metrics_df.at[node, col]
    else:
        if 'language' in G_pruned.nodes[node]:
            del G_pruned.nodes[node]['language']
        G_pruned.nodes[node]['status'] = 'not_indexed'

sample_node = list(G_pruned.nodes())[101]
attrs = G_pruned.nodes[sample_node]
print("Node attributes for", sample_node)
for k,v in attrs.items():
    print(f"-{k}: {v}")

Node attributes for https://github.com/ethereum/go-ethereum
-level: 1
-language: Go
-status: indexed
-isFork: False
-createdAt: 2013-12-26
-updatedAt: 2024-12-19
-starCount: 47933
-forkCount: 20331
-numPackages: 1
-numDependentsInOso: 238
-listOfFunders: ['Optimism' 'Gitcoin']
-totalFundingUsd: 2657310.869683773
-totalFundingUsdSince2023: 2496187.6791883335
-osoDependencyRank: 0.38208092485549133
-numReposInSameLanguage: 325
-osoDependencyRankForLanguage: 0.9907407407407407


In [7]:
G_serializable = convert_graph_to_serializable(G_pruned)
graph_json = nx.node_link_data(G_serializable)
output_path = "../graph/unweighted_graph_pruned_with_metadata.json"
with open(output_path, "w") as f:
    json.dump(graph_json, f, indent=2)